In [10]:
%%file view_deploy.py
# -*- coding: UTF-8 -*-

# author: leepand
# time: 2018-11-30
# desc: model deploy

from view_base import *
from hjs_user import *
from hjs_order import *
from hjs_ps_order import *
from bs_process import *#for service process exce_cmd
__version__ = '0.0.7'
APIFLY_FUNCTIONS="APIFLY_FUNCTIONS"
APIFLY_TOKEN="APIFLY_TOKEN"
def _run_server(apifly_function, apifly_token, host, port, workers=2, is_reg=False,static_prefix=None,
                gunicorn_opts=None):
    """
    Run the Arthur api server, wrapping it in gunicorn
    :param static_prefix: If set, the index.html asset will be served from the path static_prefix.
                          If left None, the index.html asset will be served from the root path.
    :return: None
    """
    env_map = {}
    if apifly_function:
        env_map[APIFLY_FUNCTIONS] = apifly_function
    if apifly_token:
        env_map[APIFLY_TOKEN] = apifly_token
    if static_prefix:
        env_map[STATIC_PREFIX_ENV_VAR] = static_prefix

    bind_address = "%s:%s" % (host, port)
    bind_address2 = "http//%s:%s" % (host, port)
    if is_reg:
        redis_db=api.data_api(redis_db=3).redis_api(use_type='mem',prefix='REST:SERVICE_LIST:')
        reg_info={}
        funcs_info=load_functions(apifly_function.split(","))
        addr=bind_address2
        token=apifly_token
        reg_info['addr']=addr
        reg_info['token']=token
        reg_info['funcs_list']=funcs_info
        redis_db['REST_SERVICE_REG']=json.dumps(reg_info)
    opts = shlex.split(gunicorn_opts) if gunicorn_opts else []
    exec_cmd(["gunicorn"] + opts + ["-b", bind_address, "-w", "%s" % workers,
                                    "--worker-class","gevent",
                                    "Arthur.api_server.main:app"],env=env_map, stream_output=True)
class ViewApiModelAdd(ViewBase):
    def __init__(self):
        self._rDict = {
            "apifly_function": {'n': "func", 't': str, 'v': None},
            "apifly_token": {'n': "token", 't': str, 'v': None},
            "host": {'n': "host", 't': str, 'v': ''},
            "port": {'n': "port", 't': int, 'v': None}
        }

    def _check_param(self):
        bRet, sRet = super(ViewApiModelAdd, self)._check_param()
        if not bRet:
            return False, sRet
        return True, None

    def _deploy_model_add(self):
        bRet, is_admin = HjsUser.is_admin(self.get_user_name())
        if not bRet:
            return False, sRet
        if not is_admin:
            return False, 'No permission to do this'
        return _run_server(self.func, self.token, self.host, self.port)

    def POST(self):
        bRet, sRet = self.check_login()
        if not bRet:
            return web.seeother("/login")
        bRet, sRet = self.process(self._deploy_model_add)
        if not bRet:
            Log.err("deal_order_add: %s" % (str(sRet)))
            return self.make_error(sRet)

        return self.make_response(ViewBase.RetMsg.MSG_SUCCESS)

Overwriting view_deploy.py


In [7]:

from view_base import *


ImportError: No module named bs_log

In [15]:
import requests 
response = requests.get("http://localhost:5001/experiments.json") 
print(type(response)) 
#<class 'requests.models.Response'> 
print(response.status_code) 
#200 print(type(response.text)) 
#<class 'str'> 
#response.enconding = "utf-8'
#print response.text #一长串东西...中文编码乱


<class 'requests.models.Response'>
200


In [84]:
for i in x:
    if i['name']=='benchmark-test6':
        for j in i['alternatives']:
            #print j['name']
            print j
            for k in j['data']:
                pass
                #print k

{'is_control': True, 'conversion_rate': 9.73, 'confidence_interval': 0.5327126383623012, 'name': 'default', 'participant_count': 5069, 'completed_count': 493, 'is_winner': None, 'data': [{'conversions': 265.0, 'date': '2018-10-14', 'participants': 2553.0}, {'conversions': 228.0, 'date': '2018-10-15', 'participants': 2516.0}], 'test_statistic': 'N/A', 'confidence_level': 'N/A'}
{'is_control': False, 'conversion_rate': 11.83, 'confidence_interval': 0.5912874038558433, 'name': 'alt1', 'participant_count': 4887, 'completed_count': 578, 'is_winner': None, 'data': [{'conversions': 266.0, 'date': '2018-10-14', 'participants': 2450.0}, {'conversions': 312.0, 'date': '2018-10-15', 'participants': 2437.0}], 'test_statistic': 11.45, 'confidence_level': '99.9% Confidence'}
{'is_control': False, 'conversion_rate': 9.41, 'confidence_interval': 0.5223888749951994, 'name': 'alt2', 'participant_count': 5120, 'completed_count': 482, 'is_winner': None, 'data': [{'conversions': 234.0, 'date': '2018-10-14'

In [81]:
data_date

['2018-10-11', '2018-10-12', '2018-10-13', '2018-10-14', '2018-10-15']

In [82]:
exp_name_list=[]
exp_info_list=[]

exp_series=[]
for i in x:
    if i['name']=='benchmark-test6':
        for j in i['alternatives']:
            exp_info={}
            exp_name_list.append(j['name'])
            exp_data=[0.0,0.0,0.0,0.0,0.0]
            for _iter in range(len(data_date)):
                for k in j['data']:
                    if k['date'] == data_date[_iter]:
                        print k
                        exp_data[_iter]=k['conversions']/(k['participants']+0.001)
                        #exp_data.append(k['conversions']/(k['participants']+0.001))
                    #else:
                    #    exp_data.append(0.00)
            exp_info['name']=j['name']
            exp_info['type']='line'
            exp_info['data']=exp_data
            exp_info_list.append(exp_info)

{'conversions': 265.0, 'date': '2018-10-14', 'participants': 2553.0}
{'conversions': 228.0, 'date': '2018-10-15', 'participants': 2516.0}
{'conversions': 266.0, 'date': '2018-10-14', 'participants': 2450.0}
{'conversions': 312.0, 'date': '2018-10-15', 'participants': 2437.0}
{'conversions': 234.0, 'date': '2018-10-14', 'participants': 2538.0}
{'conversions': 248.0, 'date': '2018-10-15', 'participants': 2582.0}
{'conversions': 23.0, 'date': '2018-10-14', 'participants': 2459.0}
{'conversions': 27.0, 'date': '2018-10-15', 'participants': 2484.0}


In [83]:
exp_info_list

[{'data': [0.0, 0.0, 0.0, 0.10379941096771994, 0.09061999577901599],
  'name': 'default',
  'type': 'line'},
 {'data': [0.0, 0.0, 0.0, 0.10857138425657785, 0.12802620926294245],
  'name': 'alt1',
  'type': 'line'},
 {'data': [0.0, 0.0, 0.0, 0.09219854523303969, 0.09604953677399815],
  'name': 'alt2',
  'type': 'line'},
 {'data': [0.0, 0.0, 0.0, 0.009353391885566536, 0.010869560841561657],
  'name': 'alt3',
  'type': 'line'}]

In [76]:
x=experiment_list()
date_all=[]

for i in x:
    if i['name']=='benchmark-test6':
        for j in i['alternatives']:
            print j['name']
            for k in j['data']:
                date_all.append(k['date'])

default
alt1
alt2
alt3


In [51]:
date_all

['2018-10-14',
 '2018-10-15',
 '2018-10-14',
 '2018-10-15',
 '2018-10-14',
 '2018-10-15',
 '2018-10-14',
 '2018-10-15']

In [33]:
_m

'2018-10-15'

In [78]:
#date_all=['2018-10-14','2018-10-15']
_m=max(date_all)
new_list=[]
str2date=datetime.datetime.strptime(_m,"%Y-%m-%d")
for i in range(5):
    j=i
    d_j=str2date+datetime.timedelta(days=-j)
    new_list.append(d_j.strftime("%Y-%m-%d"))
data_date=sorted(new_list)
data_date

['2018-10-11', '2018-10-12', '2018-10-13', '2018-10-14', '2018-10-15']

In [41]:
for _date in range(len(data_date)):
    print data_date[_date]

2018-10-10
2018-10-11
2018-10-12
2018-10-13
2018-10-14


In [18]:
import datetime
str="2018-08-05"
str2date=datetime.datetime.strptime(str,"%Y-%m-%d")#字符串转化为date形式
date2str=str2date.strftime("%Y%m%d")#date形式转化为str
date2str,str2date

('20180805', datetime.datetime(2018, 8, 5, 0, 0))

In [2]:
from Arthur.ABtesting.utils.models import Experiment
import Arthur.ABtesting.utils.db as db
def determine_period():
    per={'period':'day'}
    period = per.get('period', 'day')
    if period not in ['day', 'week', 'month', 'year']:
        err = {'error': 'invalid argument: {0}'.format(period), 'status': 400}
        abort(400, jsonify(err))
    return period
def simple_markdown(experiment):
    description = experiment['description']
    if description and description != '':
        experiment['pretty_description'] = markdown(description)
    return experiment
def experiment_list():
    experiments = Experiment.all(redis=db.REDIS)
    period = determine_period()
    experiments = [simple_markdown(exp.objectify_by_period(period)) for exp in experiments]
    return experiments

In [22]:
experiment_list()

[{'alternatives': [{'completed_count': 2,
    'confidence_interval': 24.633611485424034,
    'confidence_level': 'N/A',
    'conversion_rate': 33.33,
    'data': [{'conversions': 1.0, 'date': '2018-10-14', 'participants': 4.0},
     {'conversions': 1.0, 'date': '2018-10-15', 'participants': 2.0}],
    'is_control': True,
    'is_winner': None,
    'name': 'model1',
    'participant_count': 6,
    'test_statistic': 'N/A'},
   {'completed_count': 2,
    'confidence_interval': 28.043394944264506,
    'confidence_level': 'N/A',
    'conversion_rate': 40.0,
    'data': [{'conversions': 2.0, 'date': '2018-10-14', 'participants': 4.0},
     {'conversions': 0.0, 'date': '2018-10-15', 'participants': 1.0}],
    'is_control': False,
    'is_winner': None,
    'name': 'model2',
    'participant_count': 5,
    'test_statistic': 'N/A'}],
  'created_at': '2018-10-14 16:59',
  'description': None,
  'excluded_clients': 0,
  'has_winner': False,
  'is_archived': False,
  'is_paused': False,
  'kpi': N

In [23]:
from markdown import markdown

In [27]:
def archived():
    experiments = Experiment.archived(redis=db.REDIS)
    period = determine_period()
    experiments = [simple_markdown(exp.objectify_by_period(period)) for exp in experiments]
    return experiments

def paused():
    experiments = Experiment.paused(redis=db.REDIS)
    period = determine_period()
    experiments = [simple_markdown(exp.objectify_by_period(period)) for exp in experiments]#[exp.name for exp in experiments]
    return experiments
archived()

['thrift_recom_eng_abtesing3', 'test2']

In [28]:
paused()

[{'alternatives': [{'completed_count': 2,
    'confidence_interval': 24.633611485424034,
    'confidence_level': 'N/A',
    'conversion_rate': 33.33,
    'data': [{'conversions': 1.0, 'date': '2018-10-14', 'participants': 4.0},
     {'conversions': 1.0, 'date': '2018-10-15', 'participants': 2.0}],
    'is_control': True,
    'is_winner': None,
    'name': 'model1',
    'participant_count': 6,
    'test_statistic': 'N/A'},
   {'completed_count': 2,
    'confidence_interval': 28.043394944264506,
    'confidence_level': 'N/A',
    'conversion_rate': 40.0,
    'data': [{'conversions': 2.0, 'date': '2018-10-14', 'participants': 4.0},
     {'conversions': 0.0, 'date': '2018-10-15', 'participants': 1.0}],
    'is_control': False,
    'is_winner': None,
    'name': 'model2',
    'participant_count': 5,
    'test_statistic': 'N/A'}],
  'created_at': '2018-10-14 16:59',
  'description': None,
  'excluded_clients': 0,
  'has_winner': False,
  'is_archived': False,
  'is_paused': True,
  'kpi': No

In [ ]:
from Arthur.ABtesting.utils.models import Experiment
import Arthur.ABtesting.utils.db as db
def determine_period():
    per={'period':'day'}
    period = per.get('period', 'day')
    if period not in ['day', 'week', 'month', 'year']:
        err = {'error': 'invalid argument: {0}'.format(period), 'status': 400}
        abort(400, jsonify(err))
    return period
def simple_markdown(experiment):
    description = experiment['description']
    if description and description != '':
        experiment['pretty_description'] = markdown(description)
    return experiment
def experiment_list():
    experiments = Experiment.all(redis=db.REDIS)
    period = determine_period()
    experiments = [simple_markdown(exp.objectify_by_period(period)) for exp in experiments]
    return experiments